In [1]:
# Import the tables of the data set as dataframes.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

DATA_DIR = './../data' 
STUDY_DIR = DATA_DIR + '/study'


users = pd.read_csv(f'{DATA_DIR}/users.csv.gz')

# use study for less data, for testing
events = pd.read_csv(f'{STUDY_DIR}/events.csv.gz')
transactions = pd.read_csv(f'{STUDY_DIR}/transactions.csv.gz')

In [2]:
from data import preprocess_transactions, get_transactions_dataloader

train_transactions, test_transactions, user_ids, topic_ids = preprocess_transactions(transactions)

train_dataloader = get_transactions_dataloader(train_transactions, user_ids, topic_ids, 256)

test_dataloader = get_transactions_dataloader(test_transactions, user_ids, topic_ids, 256)


/home/david/ETH/EPFL/mlbd/project/recommender/data.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['topic_id'] = pd.to_numeric(df['topic_id'])


In [5]:
from data import TransactionsStudentsTopicsDS
dataset = TransactionsStudentsTopicsDS(train_transactions, user_ids, topic_ids)

In [7]:
len(dataset)

1098

In [7]:
import os
import torch
from torch import optim, nn
import lightning.pytorch as pl
from ncf_model import NCFNetwork


# TODO: fill these
num_students = len(user_ids)
num_topics = len(topic_ids)

student_embedding_dim = 32
topic_embedding_dim = 32


# init
ncf = NCFNetwork(num_students, num_topics, student_embedding_dim, topic_embedding_dim)

/home/david/anaconda3/envs/mlbd/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:197: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(


In [8]:
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model=ncf, train_dataloaders=train_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/david/anaconda3/envs/mlbd/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(

  | Name                    | Type       | Params
-------------------------------------------------------
0 | student_embedding_layer | Embedding  | 5.3 K 
1 | topic_embedding_layer   | Embedding  | 1.6 K 
2 | network                 | Sequential | 2.8 K 
3 | loss        

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
